In [7]:
import numpy as np
from copy import deepcopy
import random
INFINITY = float('inf')    
DEPTH : int = 4

In [8]:

class Board:
    def __init__(self):
        matriz = np.full((5, 5), '', dtype= '<U2')
        for i in range(0, 5):
            for j in range(0,5):
                matriz[i][j] = 0
        self.board  = matriz

    def build(self, position : tuple[int, int]):
    
        x, y = position[0], position[1]
        
        if 0 > x or x > 4 or 0 > y or y > 4:
            return False
        content :str = self.board[x,y]
        height = int(content[0])
        height = int(height)
        if height >= 4:
            return False
        height+=1
        self.board[x,y] = str(height) + content[1:]
        return True
    
    def addWorker(self, worker : str, position : tuple[int, int] ):
        if (len(self.board[position[0], position[1]]) == 1): 
            self.board[position[0], position[1]] = self.board[position[0], position[1]] + worker
            return True
        return False
    
    def WorkerPosition(self, worker : str) -> tuple[int, int]:
        for i in range(0, 5):
            for j in range(0,5):
                if worker.upper() in self.board[i,j]:
                    return [i,j]
        return None 
    
    def getWokerHeight(self, worker : str):
        pos = self.WorkerPosition(worker)
        x, y = pos[0], pos[1]
        return int(self.board[x,y][0])
    
    def WorkerAdjacentPositions(self, worker : str):
        positions : list[tuple[int, int]] = []
        position = self.WorkerPosition(worker)
        if position == None:
            return positions
        x, y = position[0], position[1]
        for i in range(-1, 2):
            for j in range(-1,2):
                newX, newY = x+i, y+j
                if((0 != i or j !=0) and x+i >= 0 and y+j >= 0 and x+i <= 4 and y+j <= 4):
                    positions.append(tuple([x+i, y+j]))
        return positions
    
    def posibleToAddWorker(self):
        posibles : list[tuple[int,int]]= []
        for i in range(0, 5):
            for j in range(0,5):
                if len(self.board[i,j]) == 1:
                    position = (i,j)
                    posibles.append(position)
        return posibles
    def validPosToMove(self, worker : str):
        ValidMoves : list[tuple[int, int]] = []
        adjacents = self.WorkerAdjacentPositions(worker)
        workerPos = self.WorkerPosition(worker)
        workerHeight = int(self.board[workerPos[0],workerPos[1]][0])
        for adj in adjacents:
            x,y = adj[0],adj[1]
            adjHeight = int(self.board[x][y][0])
            if adjHeight <= workerHeight + 1 and workerPos != [x,y]:
                ValidMoves.append(adj) 
        return ValidMoves
                                   
    def moveWorkerTo(self, worker : str, NewPosition : tuple[int, int]):
        NewPosition = tuple(NewPosition)
        if (NewPosition not in self.validPosToMove(worker)):
            return False
        for i in range(0, 5):
            for j in range(0,5):
                content = self.board[i,j]
                if worker in content:
                    self.board[i,j] = self.board[i,j][0]
        self.board[NewPosition[0],NewPosition[1]] = self.board[NewPosition[0],NewPosition[1]]  + worker
        return True
    
    def ValidPosToBuild(self, worker : str):
        ValidBuilds : list[tuple[int, int]] = []
        adjacents = self.WorkerAdjacentPositions(worker)
        workerPos = self.WorkerPosition(worker)
        for adj in adjacents:
            x,y = adj[0],adj[1]
            content = self.board[x][y]
            adjHeight = int(content[0])
            if adjHeight < 4 and len(content) == 1 :
                ValidBuilds.append(adj) 
        return ValidBuilds
    
    def startBoard(self, order : int):
        if order == 1:
            for x in [1,2]:
                notAdicionado = True
                worker = 'A' if x == 1 else 'B'
                iaWoker = 'C' if x == 1 else 'D'
                while(notAdicionado):
                    print(self.board)  
                    print(self.posibleToAddWorker() )
                    move = input('digite onde por o seu trabalhador: 1 2')
                    try:
                        position = (int(move[0]), int(move[2]))
                    except:
                        continue
                    if position in self.posibleToAddWorker() :
                        if self.addWorker(worker, position):
                            iaMove = random.choice(self.WorkerAdjacentPositions(worker))
                            if (self.addWorker(iaWoker, iaMove)):
                                notAdicionado = False
                   
        else:
            for x in [1,2]:
                notAdicionado = True
                worker = 'C' if x == 1 else 'D'
                iaWoker = 'A' if x == 1 else 'B' 
                while(notAdicionado):
                    
                    iaMove = random.choice(self.posibleToAddWorker())
                    if x == 2:
                        iaMove = random.choice(self.WorkerAdjacentPositions('C'))
                    if (self.addWorker(iaWoker, iaMove)):
                        print(self.board)  
                        print(self.posibleToAddWorker())
                        move = input('digite onde por o seu trabalhador das posições acima: ')
                        try:
                            position = (int(move[0]), int(move[2]))
                        except:
                            continue
                        if position in self.posibleToAddWorker() :
                            if self.addWorker(worker, position):
                                notAdicionado = False
    
    def startRandomBoard(self):
        workers = ['A', 'C', 'B', 'D']
        posicoes = []
        while(len(posicoes)<4):
            i = random.randint(0,4)
            j = random.randint(0,4)
            if [i,j] not in posicoes:
                posicoes.append([i,j])
        for i in range(0,4):
            self.addWorker(workers[i],posicoes[i])
            
        print('ok')
                    
class Player:
    def __init__(self, name, workers : list[str]):
        self.name = name
        self.workers = workers
    
class Node:
    def __init__(self, board : Board, maximizingPlayer : bool):
        self.board = board
        self.maximizingPlayer = maximizingPlayer
        
    


In [9]:
def get_children(node : Node):
    children : list[Node] = []
    if (node.maximizingPlayer):
        workers = [ 'A', 'B']
    else:
        workers = [ 'C', 'D']
        
    board : Board = node.board   
    auxBoard : Board
    auxBoardMoved : Board
    for worker in workers:
        for move in node.board.validPosToMove(worker): 
            auxBoard = deepcopy(board)
            auxBoard.moveWorkerTo(worker, move)
            for build in auxBoard.ValidPosToBuild(worker):
                auxBoardMoved = deepcopy(auxBoard)                  
                auxBoardMoved.build(build)
                otherPlayer = not node.maximizingPlayer
                auxNode = Node(auxBoardMoved, otherPlayer)
                children.append(auxNode)
    return children

def evaluate(node: Node):
    """
    Avalia o estado atual do jogo.
    Retorna um valor positivo se for favorável ao jogador maximizador,
    e negativo se for favorável ao jogador minimizador.
    """
    board = node.board
    workers_max = ['A', 'B']  # Trabalhadores do jogador maximizador
    workers_min = ['C', 'D']  # Trabalhadores do jogador minimizador

    max_score = 0
    min_score = 0

    
    for worker in workers_max:
        position = board.WorkerPosition(worker)
        if position is not None:
            x, y = position
            height = int(board.board[x, y][0])
            if(height == 3):
                max_score += 1000
            max_score += height 

  
    for worker in workers_min:
        position = board.WorkerPosition(worker)
        if position is not None:
            x, y = position
            height = int(board.board[x, y][0])
            if(height == 3):
                min_score += 1000
            min_score += height
    return max_score - min_score 

def isTerminal(node : Node):
    workers =  ['A', 'B'] if node.maximizingPlayer else [ 'C', 'D']
    for worker in ['A', 'B', 'C', 'D']:
        height = node.board.getWokerHeight(worker)
        if height == 3:
            return True 
    if (node.board.validPosToMove('A') == [] and node.board.validPosToMove('B') == []) or (node.board.validPosToMove('C') == [] and node.board.validPosToMove('D') ==[]):
        return True
    return False

def minimax_alpha_beta(node, depth, alpha, beta, maximizing_player, evaluate, get_children):
    if depth == 0 or not get_children(node):
        return evaluate(node), None
    
    if isTerminal(node):
        return evaluate(node), node
    
    move : Node = node
    if maximizing_player:
        max_eval = -INFINITY
       
        for child in get_children(node):
            eval, _ = minimax_alpha_beta(child, depth - 1, alpha, beta, False, evaluate, get_children)
            if eval > max_eval:
                max_eval = max(max_eval, eval)
                move = child
                alpha = max(alpha, eval)
            if beta <= alpha:
                break
        return max_eval , move
    else:
        min_eval = INFINITY
        for child in get_children(node):
            eval, _ = minimax_alpha_beta(child, depth - 1, alpha, beta, True, evaluate, get_children)
            if eval < min_eval:
                min_eval = min(min_eval, eval)
                move = child
                beta = min(beta, eval)
            if beta <= alpha:
                break
        return min_eval , move
        
    
def randomPlayer(node : Node):
    childs = get_children(node)
    randomMove = random.choice(childs)
    return randomMove

def iaPlayer(node : Node):
    _, bestMove = minimax_alpha_beta(node, DEPTH, -INFINITY, INFINITY, node.maximizingPlayer, evaluate, get_children)
    return bestMove


In [10]:
def manualPlayer(node : Node):
    moves : list[tuple[int, int]] = []
    workersNames = []
    auxBoard = deepcopy(node.board)
    worker = ''
    if node.maximizingPlayer:
        workersNames = ['A','B']
    else:
        workersNames = ['C', 'D']
    
    moveMade = True    
    while moveMade: 
        invalido = True
        while invalido:
            print('Escolha dentre os trabalhadores a seguir qual voce deseja mover: ')
            print(workersNames)
            worker = input('digite apenas a letra que representa o trabalhador. ex: A').upper()
            if worker in workersNames:
                moves = node.board.validPosToMove(worker)
                if len(moves)> 0:
                    invalido = False   
            else:
                print('VOCE DIGITOU UM TRABALHADOR INVALIDO')
        print('''Escolha dentre os movimentos a seguir para onde voce deseja andar \n
            sendo o primeiro valor a linha no tabuleiro e o segundo a coluna\n
            leve em consideração que o valor da primeira linha/coluna é 0\n
            digite linha e coluna eparados por um espaço, por exemplo, para a linha 0 coluna 3, digite: 0 3''')
        print(moves)
        myMove : tuple[int,int] = (int(10),int(10))
        while not( myMove in moves):
            print('DIGITE SEU MOVIMENTO: ')
            myMove = input('digite seu movimento ex: 1 2')
            try:
                x, y = int(myMove[0]), int(myMove[2])
                myMove = (x, y)
            except:
                print('MOVIMENTO INVALIDO')
                
        moveMade = not auxBoard.moveWorkerTo(worker, myMove)
    buildMade = True  
    builds = auxBoard.ValidPosToBuild(worker)            
    while buildMade:
        print('''Escolha dentre as posições a seguir onde voce deseja construir\n
        sendo o primeiro valor a linha no tabuleiro e o segundo a coluna\n
        leve em consideração que o valor da primeira linha/coluna é 0\n
        digite linha e coluna eparados por um espaço, por exemplo, para a linha 0 coluna 3, digite: 0 3''' )
        print(builds)
        myBuild : tuple[int,int] = (int(10),int(10))
        while not( myBuild in builds):
            print('DIGITE A POSIÇÃO PARA CONSTRUIR: ')
            myBuild = input('ex: 1 2')
            try:
                x, y = int(myBuild[0]), int(myBuild[2])
                myBuild = (x, y)
            except:
                print('MOVIMENTO INVALIDO')
                
        buildMade = not auxBoard.build(myBuild)            
    node.board, node.maximizingPlayer = auxBoard  , not node.maximizingPlayer
    return node

In [ ]:
def definePlayers():
    name = input('Digite o seu nome: ')
    order : int = 0
    while order not in [1,2]:
        try:
            order = int(input('Digite a ordem para voce jogar, se primeiro digite 1, se segundo a jogar digite 2: ').split()[0])
        except:
            print('ordem invalida')
            order = int(input('Digite a ordem para voce jogar, se primeiro digite 1, se segundo a jogar digite 2: ').split()[0])
    
    player1Name = name
        
    return order, player1Name


def main():
    order = 1
    player1 = 'IA'
    #order, player1 = definePlayers()
    
    BOARD = Board()
    print('inicio:')
    print(BOARD.board)
    print('adicionado:')
    #BOARD.startBoard(order)
    BOARD.startRandomBoard()
    print(BOARD.board)
    GAME : Node = Node(BOARD, True)
    
    
    
    if order == 1: 
        while not isTerminal(GAME):
            if GAME.maximizingPlayer:
                print(f'PLAYER 1 {player1} JOGANDO....')
                GAME = iaPlayer(GAME)
                #GAME = manualPlayer(GAME)
                print(GAME.board.board)
                if (isTerminal(GAME)):
                    continue
            else:
                print('PLAYER 2 (IA) JOGANDO....')
                GAME = iaPlayer(GAME)
                print(GAME.board.board)
    else: 
        while not isTerminal(GAME):
            if GAME.maximizingPlayer:
                print(f'PLAYER 1 (IA) JOGANDO....')
                GAME = iaPlayer(GAME)
                print(GAME.board.board)
                if (isTerminal(GAME)):
                    continue
            else:
                print(f'PLAYER 2 {player1} JOGANDO....')
                GAME = iaPlayer(GAME)
                #GAME = manualPlayer(GAME)
                
                print(GAME.board.board)
    vencedor = 'player 1' if not GAME.maximizingPlayer else 'player 2'    
    print(f'JOGO FINALIZADO. VENCEDOR(A): {vencedor}')
main()
